In [1]:
import findspark
findspark.init('D:\spark-2.3.3-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
print('Modules Imported')

Modules Imported


In [2]:
spark=SparkSession.builder.appName('cruise').getOrCreate()

In [6]:
df=spark.read.csv('D:\python coding\pyspark_tutorial\Linear regression\cruise_ship_info.csv',inferSchema=True,header=True)


In [8]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [9]:
for item in df.head(5):
    print(item)
    print('\n')

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)


Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)


Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7)


Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1)


Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0)




In [10]:
df.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [12]:
#to convert string into a numeric feature use stringinddexer
from pyspark.ml.feature import StringIndexer

In [17]:
indexer=StringIndexer(inputCol='Cruise_line',outputCol='cruise_cat')
indexed=indexer.fit(df).transform(df)
indexed.head(3)
#indexed.select('Cruise_line','cruise_cat').show()

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0)]

In [18]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler


In [21]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cat']

In [27]:
#creating vectors now
assembler=VectorAssembler(inputCols=['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'cruise_cat'],outputCol='features')
output=assembler.transform(indexed)

output.select('features','crew').show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [28]:
final_data=output.select('features','crew')

In [29]:
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [30]:
train_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              110|
|   mean|8.036545454545466|
| stddev|3.494682050706418|
|    min|              0.6|
|    max|             21.0|
+-------+-----------------+



In [31]:
from pyspark.ml.regression import LinearRegression

In [32]:
ship_lr=LinearRegression(featuresCol='features',labelCol='crew')

In [33]:
trained_ship_model=ship_lr.fit(train_data)

In [35]:
ship_results=trained_ship_model.evaluate(train_data)

In [38]:
ship_results.rootMeanSquaredError

0.9339423911007266

In [39]:
ship_results.r2

0.9279239421964878

In [40]:
from pyspark.sql.functions import corr

In [45]:
#checking the correlation between columns
#correlation is always between tow columns
df.select(corr('crew','passengers')).show()


+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [46]:
#testing model on unlabeled data
unlabeled_data=test_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[6.0,30.276999999...|
|[6.0,93.0,23.94,9...|
|[6.0,158.0,43.7,1...|
|[7.0,89.6,25.5,9....|
|[9.0,90.09,25.01,...|
|[9.0,113.0,26.74,...|
|[10.0,58.825,15.6...|
|[10.0,81.76899999...|
|[10.0,90.09,25.01...|
|[10.0,105.0,27.2,...|
|[11.0,58.6,15.66,...|
|[11.0,90.09,25.01...|
|[11.0,91.62700000...|
|[11.0,138.0,31.14...|
|[12.0,58.6,15.66,...|
|[12.0,88.5,21.24,...|
|[12.0,88.5,21.24,...|
|[12.0,91.0,20.32,...|
|[12.0,91.0,20.32,...|
|[12.0,138.0,31.14...|
+--------------------+
only showing top 20 rows



In [49]:
predictions=trained_ship_model.transform(unlabeled_data)
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[6.0,30.276999999...| 4.290267415583632|
|[6.0,93.0,23.94,9...|10.561152805966858|
|[6.0,158.0,43.7,1...|14.018897542771231|
|[7.0,89.6,25.5,9....|11.051686362634006|
|[9.0,90.09,25.01,...| 9.355005994457068|
|[9.0,113.0,26.74,...|11.363942765011508|
|[10.0,58.825,15.6...| 7.295815705089378|
|[10.0,81.76899999...| 8.805872457623725|
|[10.0,90.09,25.01...| 8.981103875979219|
|[10.0,105.0,27.2,...|11.252031636345482|
|[11.0,58.6,15.66,...| 7.420574090726334|
|[11.0,90.09,25.01...| 8.967827505094366|
|[11.0,91.62700000...|  9.21842095113343|
|[11.0,138.0,31.14...|12.994946388000683|
|[12.0,58.6,15.66,...|  7.41112032911233|
|[12.0,88.5,21.24,...| 9.512079350079523|
|[12.0,88.5,21.24,...|10.380859560189924|
|[12.0,91.0,20.32,...| 9.179002757761216|
|[12.0,91.0,20.32,...| 9.179002757761216|
|[12.0,138.0,31.14...| 12.98167001711583|
+--------------------+------------